In [1]:
from Annotation_helper import *
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler

In [2]:
matches = pd.read_csv("Matches/Bas2.csv", index_col=0)

In [3]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
# transactions = transactions.set_index("ID")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")
# permissions = permissions.set_index("ID")

In [4]:
distances = pd.read_csv("Distances/distances_0.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
for i in range(1, 99):
    df2 = pd.read_csv(f"Distances/distances_{i}.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
    distances = pd.concat([distances, df2])

In [5]:
distances

,permission_indx,transaction_indx,SlaafGender,SlaafNaamNieuw,BezitterGender,BezitterVoornaam,BezitterBeroep,BezitterAchternaam
0,0,0,True,0.000000,False,0.000000,0.0,0.000000
1,0,1,True,0.250000,False,0.000000,0.0,0.000000
2,0,2,False,0.166626,False,0.000000,0.0,0.000000
3,0,3,True,0.111084,False,0.000000,0.0,0.000000
4,0,4,True,0.142822,False,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...
265951,3889,7192,True,0.250000,True,0.166626,0.0,0.166626
265952,3889,7193,False,0.250000,True,0.166626,0.0,0.166626
265953,3889,7194,False,0.000000,True,0.277832,0.0,0.000000
265954,3889,7195,False,0.000000,True,0.166626,0.0,0.166626


In [6]:
matches

,Annotator,transaction_indx,permission_indx
0,Bas,None,198
1,Bas,None,199
2,Bas,None,200
3,Bas,None,201
4,Bas,None,202
...,...,...,...
225,Bas,None,286
226,Bas,None,287
227,Bas,None,288
228,Bas,None,198


In [7]:
for match in matches.iterrows():
    t = pd.Series(0, index=transactions["Entry-ID"])
    break

In [8]:
t

Entry-ID
0       0
1       0
2       0
3       0
4       0
       ..
7192    0
7193    0
7194    0
7195    0
7196    0
Length: 7188, dtype: int64

In [9]:
transactions["Entry-ID"]

0          0
1          1
2          2
3          3
4          4
        ... 
7192    7192
7193    7193
7194    7194
7195    7195
7196    7196
Name: Entry-ID, Length: 7188, dtype: int64

In [10]:
trans_ID = list(transactions["Entry-ID"])

In [11]:
res = dict()
for i, match in matches.iterrows():
    res[i] = {"transaction_indx" : match["transaction_indx"], "permission_indx" : match["permission_indx"], "match": 1}

In [12]:
res

{0: {'transaction_indx': 'None', 'permission_indx': 198, 'match': 1},
 1: {'transaction_indx': 'None', 'permission_indx': 199, 'match': 1},
 2: {'transaction_indx': 'None', 'permission_indx': 200, 'match': 1},
 3: {'transaction_indx': 'None', 'permission_indx': 201, 'match': 1},
 4: {'transaction_indx': 'None', 'permission_indx': 202, 'match': 1},
 5: {'transaction_indx': 'None', 'permission_indx': 203, 'match': 1},
 6: {'transaction_indx': 'None', 'permission_indx': 204, 'match': 1},
 7: {'transaction_indx': 'None', 'permission_indx': 205, 'match': 1},
 8: {'transaction_indx': 'None', 'permission_indx': 206, 'match': 1},
 9: {'transaction_indx': '6099', 'permission_indx': 207, 'match': 1},
 10: {'transaction_indx': '4149', 'permission_indx': 208, 'match': 1},
 11: {'transaction_indx': 'None', 'permission_indx': 209, 'match': 1},
 12: {'transaction_indx': '4159', 'permission_indx': 210, 'match': 1},
 13: {'transaction_indx': '4162', 'permission_indx': 210, 'match': 1},
 14: {'transacti

In [13]:
pd.DataFrame.from_dict({(i,j):res[i][j] 
                           for i in res.keys() 
                           for j in res[i].keys()},
                       orient='index')

,0
"(0, transaction_indx)",None
"(0, permission_indx)",198
"(0, match)",1
"(1, transaction_indx)",None
"(1, permission_indx)",199
...,...
"(228, permission_indx)",198
"(228, match)",1
"(229, transaction_indx)",None
"(229, permission_indx)",199


In [14]:
matches["permission_indx"].unique()

array([198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
       211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
       224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236,
       237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
       250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262,
       263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275,
       276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288,
       289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301,
       302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314,
       315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327,
       328, 329, 330, 331, 332], dtype=int64)

In [15]:
X = distances[distances['permission_indx'].isin(matches["permission_indx"].unique())]

In [16]:
X["Match"] = 0

<ipython-input-16-1ce1c902cb90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Match"] = 0


In [17]:
for i, match in matches.iterrows():
    if match["transaction_indx"] != "None":
        indx = distances.query(f"permission_indx == {match['permission_indx']} & transaction_indx == {match['transaction_indx']}").index[0]
        X.iloc[indx, -1] = 1

C:\Users\basvd\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
data = X.iloc[:, 2:-1]
y = X.iloc[:, -1]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.5, random_state=42)

In [ ]:
# (tn, fp,
# fn, tp)

# Benchmarks

## All false

In [ ]:
pred = np.zeros(len(y_test))

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Normal sampling

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB().fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

## Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50, 20), random_state=1).fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

# Oversampling

In [ ]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train, y_train)

## Logistic regression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_ros, y_ros)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB().fit(X_ros, y_ros)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

## Neural network

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_ros, y_ros)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

# Smart Oversampling

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X_train, y_train)

## Logistic regression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_res, y_res)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

## Neural network

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_res, y_res)
pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

In [20]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
X_scaler = MinMaxScaler().fit(X_train.to_numpy().astype(float))
X_train = X_scaler.transform(X_train.to_numpy().astype(float))

In [23]:
[y_train.to_numpy().astype(float).T]

[array([0., 0., 0., ..., 0., 0., 0.])]

In [24]:
y_scaler = MinMaxScaler().fit([y_train.to_numpy().astype(float).T])
# y_train = y_scaler.transform([y_train.to_numpy().astype(float).T])

In [25]:
X_test = X_scaler.transform(X_test.astype(float))
# y_scaler = MinMaxScaler().fit([y_test.astype(float).T])
# y_test = y_scaler.transform([y_test.astype(float).T])

C:\Users\basvd\anaconda3\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [26]:
# X_scaler = MinMaxScaler().fit(X_test.astype(float))
# y_scaler = MinMaxScaler().fit([y_test])
# X_test = X_scaler.transform(X_test.astype(float))
# y_test = y_scaler.transform([y_test.astype(float)])

In [27]:
# X_scaler.transform(X_train.to_numpy().astype(float))

In [41]:
y_train.values.astype(float)

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
y_test.values.astype(float).max()

1.0

In [28]:
y_train.min()

0

In [29]:
y_train.max()

1

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation='softmax'),
    tf.keras.layers.Dense(1)
])

In [36]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [51]:
y_train.values.astype(float).T

array([0., 0., 0., ..., 0., 0., 0.])

In [47]:
model.fit(X_train, y_train.values.astype(float).T, epochs=1, batch_size=1000)

486/486 [==============================] - 2s 5ms/step - loss: nan - accuracy: 1.0000


In [ ]:
model.predict(X_train[:2])

In [ ]:
y_train.min()

In [ ]:
y_test.max()

In [45]:
pred = model.predict(X_test)

In [ ]:
y_test[0]

In [ ]:
pred.T[0].max()

In [46]:
cm = confusion_matrix(y_test.values.astype(float), np.round(pred.T[0]))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()